In [6]:
# !pip install -r requirements.txt

In [38]:
from platform import python_version

from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from langchain_openai import OpenAIEmbeddings
from qdrant_client.models import VectorParams, Distance
from dotenv import load_dotenv
load_dotenv()
import os

from typing import List, Optional
import os
from pathlib import Path

from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.agents import create_agent

### альтернативный вариант

In [8]:
from platform import python_version
import os
from pathlib import Path
from typing import List, Optional

from dotenv import load_dotenv
load_dotenv()

import re
import uuid
import pandas as pd

from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance

from langchain_qdrant import QdrantVectorStore
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter, TokenTextSplitter

import clickhouse_connect

# (если понадобится) from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader


In [9]:
# 2.1 Настройка подключения к Qdrant
QDRANT_URL = os.getenv("QDRANT_URL")
client_qdrant = QdrantClient(url=QDRANT_URL)

# 2.2 Настройка подключения к ClickHouse
CH_HOST = '84.201.160.255'   # или из окружения
CH_PORT = 8123
CLICKHOUSE_USER = 'peter'
CLICKHOUSE_PASSWORD = '1234'

client_clickhouse = clickhouse_connect.get_client(
    host=CH_HOST,
    port=CH_PORT,
    username=CLICKHOUSE_USER,
    password=CLICKHOUSE_PASSWORD
)


In [10]:
emails_df = client_clickhouse.query_df("""
    SELECT id, message_id, subject, from_addr, to_addr, cc_addr, bcc_addr,
           sent_at_utc, folder, body_text, body_html
    FROM mailkb.emails
    ORDER BY sent_at_utc DESC, message_id DESC
    --LIMIT 100
""")


Настройка коллекции в Qdrant (с вариантами)

In [11]:
BASE_URL = "http://localhost:8000/v1"
embeddings = OpenAIEmbeddings(
    # model="Qwen/Qwen3-Embedding-8B",
    model="Qwen/Qwen3-Embedding-0.6B",
    api_key="not-needed",
    base_url=BASE_URL,
    tiktoken_enabled=False,
)

In [12]:
# тест - получаем размерность эмбединга
vec = embeddings.embed_query("test")
EMBEDDING_DIM = len(vec)
EMBEDDING_DIM

1024

In [13]:
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance
from langchain_qdrant import QdrantVectorStore

COLLECTION_NAME = "mailkb_emails"


# ---------- Вариант A: УДАЛИТЬ коллекцию и СОЗДАТЬ ЗАНОВО ----------
def setup_collection_recreate():
    """
    Полная пересоздача коллекции: удаляем старую, создаём новую с нужной размерностью.
    Полезно, когда меняешь модель / размер эмбеддинга.
    """
    client_qdrant.recreate_collection(
        collection_name=COLLECTION_NAME,
        vectors_config=VectorParams(
            size=EMBEDDING_DIM,
            distance=Distance.COSINE,
        ),
    )

    qv = QdrantVectorStore(
        client=client_qdrant,
        collection_name=COLLECTION_NAME,
        embedding=embeddings,
        # если не хочешь, чтобы он дополнительно валидировал конфиг:
        # validate_collection_config=False,
    )
    return qv


# ---------- Вариант B: СОЗДАТЬ, если коллекции ещё нет ----------
def setup_collection_create_if_not_exists():
    """
    Если коллекции нет — создаём.
    Если есть — НЕ трогаем (данные сохраняются).
    """
    collections = client_qdrant.get_collections().collections
    existing_names = {c.name for c in collections}

    if COLLECTION_NAME not in existing_names:
        client_qdrant.create_collection(
            collection_name=COLLECTION_NAME,
            vectors_config=VectorParams(
                size=EMBEDDING_DIM,
                distance=Distance.COSINE,
            ),
        )

    qv = QdrantVectorStore(
        client=client_qdrant,
        collection_name=COLLECTION_NAME,
        embedding=embeddings,
        # при желании можно отключить проверку:
        # validate_collection_config=False,
    )
    return qv


# ---------- Вариант C: ИСПОЛЬЗОВАТЬ ТЕКУЩУЮ коллекцию как есть ----------
def setup_collection_use_existing():
    """
    Просто подключаемся к уже существующей коллекции.
    Ничего не создаём и не удаляем.
    Важно: размерность в Qdrant должна совпадать с размерностью эмбеддинга.
    """
    qv = QdrantVectorStore(
        client=client_qdrant,
        collection_name=COLLECTION_NAME,
        embedding=embeddings,
        # если вдруг Qdrant создан с "левым" размером, и ты осознанно хочешь
        # отключить проверку (НЕ рекомендую в бою):
        # validate_collection_config=False,
    )
    return qv



In [14]:
# Пример использования одного из вариантов:
# qv = setup_collection_recreate()
qv = setup_collection_create_if_not_exists()
# qv = setup_collection_use_existing()

Подготовка утилит (нормализация, участники, превращение в документы)

In [15]:
RE_PREFIX = re.compile(r'^\s*(re|fw|fwd):\s*', flags=re.IGNORECASE)

def normalize_subject(subj: str) -> str:
    s = subj or ""
    while True:
        ns = RE_PREFIX.sub('', s).strip()
        if ns == s:
            break
        s = ns
    s = re.sub(r'\s+', ' ', s)
    return s.lower()

def participants_list(row) -> list[str]:
    def _norm(x):
        if not x:
            return []
        if isinstance(x, list):
            return [str(i).strip() for i in x if str(i).strip()]
        return [p.strip() for p in re.split(r'[;,]', str(x)) if p.strip()]
    people = _norm(row.get("from_addr")) + _norm(row.get("to_addr")) \
           + _norm(row.get("cc_addr")) + _norm(row.get("bcc_addr"))
    return sorted(set(people))

def build_docs(df: pd.DataFrame) -> list[Document]:
    docs: list[Document] = []
    for _, r in df.iterrows():
        subj = (r.get("subject") or "").strip()
        body = (r.get("body_text") or "").strip()
        if not body:
            body = (r.get("body_html") or "").strip()
        text = (subj + "\n\n" + body).strip()
        if not text:
            continue
        parts = participants_list(r)
        norm_subj = normalize_subject(subj)
        thread_key = f"{norm_subj}||{';'.join(sorted(parts))}"
        meta = {
            "row_id": r.get("id"),
            "message_id": r.get("message_id"),
            "subject": subj,
            "sent_at_utc": str(r.get("sent_at_utc")),
            "folder": r.get("folder"),
            "from_addr": parts[:1],
            "participants": parts,
            "thread_key": thread_key,
        }
        docs.append(Document(page_content=text, metadata=meta))
    return docs


Очистка текста и разбивка на чанки

In [16]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1200, chunk_overlap=150)

RE_QUOTED = re.compile(r"(?m)^(>+).*$")  # строки, начинающиеся с >
RE_HDR = re.compile(r"(?:^|\n)(from:|sent:|to:|subject:).*(?:\n.*){0,20}", re.IGNORECASE)

def clean_text(t: str) -> str:
    t = t.replace("\r\n", "\n")
    t = RE_HDR.sub("\n", t)
    t = RE_QUOTED.sub("", t)
    t = "\n".join([ln.strip() for ln in t.split("\n") if ln.strip()])
    return t

def preprocess_and_chunk(docs: list[Document]) -> list[Document]:
    cleaned: list[Document] = []
    for d in docs:
        txt = clean_text(d.page_content)
        if not txt:
            continue
        cleaned.append(Document(page_content=txt, metadata=d.metadata))
    return splitter.split_documents(cleaned)


Генерация стабильных ID и загрузка в Qdrant

In [17]:
def make_ids(chunks: list[Document]) -> list[str]:
    counters: dict[str, int] = {}
    ids: list[str] = []
    for c in chunks:
        msg = c.metadata.get("message_id") or c.metadata.get("row_id") or "noid"
        i = counters.get(msg, 0)
        counters[msg] = i + 1
        raw = f"{msg}::chunk_{i}"
        uid = uuid.uuid5(uuid.NAMESPACE_URL, raw)
        ids.append(str(uid))
    return ids

def process_batch(df: pd.DataFrame):
    docs = build_docs(df)
    chunks = preprocess_and_chunk(docs)
    if not chunks:
        return
    ids = make_ids(chunks)
    texts = [c.page_content for c in chunks]
    metadatas = [c.metadata for c in chunks]
    qv.add_texts(texts=texts, metadatas=metadatas, ids=ids)


Запуск загрузки (батчинг)

In [18]:
# полный цикл с батчингом
# BATCH_SIZE = 5000
# offset = 0
#
# while True:
#     df_batch = client_clickhouse.query_df(f"""
#         SELECT id, message_id, subject, from_addr, to_addr, cc_addr, bcc_addr,
#                sent_at_utc, folder, body_text, body_html
#         FROM mailkb.emails
#         ORDER BY sent_at_utc ASC
#         LIMIT {BATCH_SIZE} OFFSET {offset}
#     """)
#     if df_batch.empty:
#         break
#     process_batch(df_batch)
#     offset += len(df_batch)
#     print(f"Processed {offset} rows")

# Или — одноразовая загрузка небольшой выборки
offset = 0
df_small = client_clickhouse.query_df("""
    SELECT id, message_id, subject, from_addr, to_addr, cc_addr, bcc_addr,
           sent_at_utc, folder, body_text, body_html
    FROM mailkb.emails
    ORDER BY sent_at_utc DESC, message_id DESC
    LIMIT 100
""")
process_batch(df_small)
offset += len(df_small)


Аналитика переписки (треды, темы, теги проектов)

In [19]:
from collections import Counter
# Утилиты
def split_addrs(x) -> list[str]:
    if x is None:
        return []
    if isinstance(x, list):
        return [a.strip() for a in x if str(a).strip()]
    return [a.strip() for a in re.split(r'[;,]', str(x)) if a.strip()]

def participants(row) -> list[str]:
    lst = split_addrs(row.get("from_addr")) + split_addrs(row.get("to_addr")) \
        + split_addrs(row.get("cc_addr")) + split_addrs(row.get("bcc_addr"))
    return sorted(set(lst))

def clean_for_topics(text: str) -> list[str]:
    if not text:
        return []
    t = text.lower()
    t = re.sub(r'[^a-zа-я0-9\s\-]+', ' ', t)
    t = re.sub(r'\s+', ' ', t).strip()
    toks = t.split()
    stop = set("""
        и в во на с со от до по за для при как что это это/that of the a an to is are was were be been being
        у о об обo про из из-за над под между но или либо либо/или который которые какая какие чей чья чей-то
        re fw fwd subject тема письмо письма письме по- поводу
    """.split())
    return [w for w in toks if len(w) > 2 and w not in stop]

PROJECT_KEYWORDS = {
    "segezha": "Проект: Segezha",
    "ewm": "SAP EWM",
    "bw": "SAP BW",
    "o2c": "O2C",
    "mnf": "MNF",
    "сцбк": "СЦБК",
    "вагон": "Логистика/Вагоны",
}

def guess_project_tags(subject: str, body: str, addrs: list[str]) -> list[str]:
    text = f"{subject or ''} {body or ''}".lower()
    tags = set()
    for kw, tag in PROJECT_KEYWORDS.items():
        if kw in text:
            tags.add(tag)
    for a in addrs:
        m = re.search(r'@([a-z0-9\.-]+)', a.lower())
        if m:
            dom = m.group(1)
            if 'segezha' in dom:
                tags.add("Проект: Segezha")
            if 'bearingpoint' in dom:
                tags.add("Внутренние/Подрядчик")
    return sorted(tags)

# Добавление колонок в df
df = emails_df.copy()
df["norm_subject"] = df["subject"].apply(lambda s: normalize_subject(s if isinstance(s, str) else ""))
df["participants"] = df.apply(participants, axis=1)
df["thread_key"] = df.apply(lambda r: f'{r["norm_subject"]}||{";".join(r["participants"])}', axis=1)

def plain_body(row):
    t = (row.get("body_text") or "").strip()
    if t:
        return t
    return (row.get("body_html") or "").strip()

df["plain_body"] = df.apply(plain_body, axis=1)
df["topics"] = df.apply(lambda r: clean_for_topics(r["subject"]) + clean_for_topics(r["plain_body"][:1000]), axis=1)
df["project_tags"] = df.apply(lambda r: guess_project_tags(r["subject"], r["plain_body"], r["participants"]), axis=1)

threads = (
    df.groupby("thread_key")
      .agg(
         first_sent=("sent_at_utc","min"),
         last_sent=("sent_at_utc","max"),
         n_emails=("id","count"),
         subjects=("subject", lambda s: list(pd.unique([x for x in s if isinstance(x, str)]))[:5]),
         participants=("participants", lambda cols: sorted(set(sum(cols, [])))),
         projects=("project_tags", lambda cols: sorted(set(sum(cols, [])))),
         topics=("topics", lambda cols: [w for w,_ in Counter(sum(cols, [])).most_common(10)])
      )
      .reset_index()
      .sort_values(["last_sent","n_emails"], ascending=[False, False])
)

def flatten(col):
    out = []
    for arr in col:
        out.extend(arr)
    return out

projects_df = (
    threads.assign(project=lambda x: x["projects"].apply(lambda arr: arr if arr else ["(Не классифицировано)"]))
           .explode("project")
           .groupby("project")
           .agg(
              n_threads=("thread_key","nunique"),
              n_emails=("n_emails","sum"),
              first_sent=("first_sent","min"),
              last_sent=("last_sent","max"),
              participants=("participants", lambda cols: sorted(set(flatten(cols)))[:50]),
              top_topics=("topics", lambda cols: [w for w,_ in Counter(flatten(cols)).most_common(15)])
           )
           .reset_index()
           .sort_values(["n_threads","n_emails"], ascending=[False, False])
)


C:\Users\Peter\AppData\Local\Temp\ipykernel_29396\3129374694.py:77: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  subjects=("subject", lambda s: list(pd.unique([x for x in s if isinstance(x, str)]))[:5]),


Сводка тредов с помощью LLM

### Первый агент

In [20]:
import json
from collections import defaultdict
from langchain.tools import tool


@tool
def search_project_emails(project_hint: str, limit: int = 200) -> str:
    """
    Найти и сгруппировать письма по проекту/теме.

    Вход:
        project_hint: текстовая подсказка — название проекта, код ЗНИ, ключевые слова.
        limit: максимум писем для выборки.

    Логика:
      1) делаем семантический поиск по Qdrant;
      2) группируем результаты по thread_key;
      3) сортируем письма внутри треда по sent_at_utc.

    Выход:
        JSON-строка со структурой:
        {
          "project_hint": "...",
          "threads": [
            {
              "thread_key": "...",
              "subject": "...",
              "participants": [...],
              "messages": [
                {
                  "message_id": "...",
                  "sent_at_utc": "...",
                  "from_addr": "...",
                  "snippet": "...",
                  "subject": "...",
                  "folder": "...",
                  "participants": [...],
                  "metadata": {...}
                }
              ]
            }
          ]
        }
    """
    # 1. Семантический поиск по тексту
    docs = qv.similarity_search(project_hint, k=limit)

    threads = defaultdict(list)

    for doc in docs:
        md = doc.metadata or {}

        thread_key = md.get("thread_key") or md.get("subject") or "unknown_thread"

        message = {
            "row_id": md.get("row_id"),
            "message_id": md.get("message_id"),
            "sent_at_utc": md.get("sent_at_utc"),
            "from_addr": (md.get("from_addr") or [None])[0],
            "subject": md.get("subject"),
            "folder": md.get("folder"),
            "participants": md.get("participants") or [],
            # обрезаем тело, чтобы не заливать в ответ мегатекст
            "snippet": (doc.page_content[:600] + "…") if doc.page_content else None,
            "metadata": md,
        }

        threads[thread_key].append(message)

    thread_list = []
    for thread_key, messages in threads.items():
        # сортируем по дате
        messages_sorted = sorted(
            messages,
            key=lambda m: m.get("sent_at_utc") or "",
        )

        # subject — из какого-то письма (обычно это RE:/FW: и т.п.)
        subject = None
        for m in messages_sorted:
            if m.get("subject"):
                subject = m["subject"]
                break

        # участники: агрегируем по всем письмам
        participants_set = set()
        for m in messages_sorted:
            for p in m.get("participants") or []:
                participants_set.add(p)

        thread_list.append(
            {
                "thread_key": thread_key,
                "subject": subject,
                "participants": sorted(participants_set),
                "messages": messages_sorted,
            }
        )

    result = {
        "project_hint": project_hint,
        "threads": thread_list,
    }

    return json.dumps(result, ensure_ascii=False, indent=2)


In [21]:
@tool
def search_emails_raw(query: str, limit: int = 50) -> str:
    """
    Общий поиск писем в Qdrant по любому запросу (без группировки).
    Удобно для отладки.
    """
    docs = qv.similarity_search(query, k=limit)
    data = []
    for doc in docs:
        md = doc.metadata or {}
        data.append({
            "thread_key": md.get("thread_key"),
            "row_id": md.get("row_id"),
            "message_id": md.get("message_id"),
            "subject": md.get("subject"),
            "sent_at_utc": md.get("sent_at_utc"),
            "from_addr": (md.get("from_addr") or [None])[0],
            "participants": md.get("participants") or [],
            "folder": md.get("folder"),
            "snippet": (doc.page_content[:400] + "…") if doc.page_content else None,
        })
    return json.dumps({"query": query, "results": data}, ensure_ascii=False, indent=2)

In [22]:
@tool
def search_project_emails_batch(project_hint: str, offset: int = 0, batch_size: int = 50) -> str:
    """
    Возвращает batch данных по offset.
    """
    docs = qv.similarity_search(project_hint, k=offset + batch_size)
    docs = docs[offset: offset + batch_size]

    batch = []
    for d in docs:
        md = d.metadata or {}
        batch.append({
            "subject": md.get("subject"),
            "snippet": d.page_content[:600],
            "sent_at_utc": md.get("sent_at_utc"),
            "thread_key": md.get("thread_key"),
        })

    return json.dumps({
        "project_hint": project_hint,
        "offset": offset,
        "batch_size": batch_size,
        "batch_len": len(batch),
        "has_more": len(batch) == batch_size,
        "batch": batch
    }, ensure_ascii=False)


In [23]:
from pathlib import Path
import json
from langchain.tools import tool

SUMMARY_DIR = Path("summaries")
SUMMARY_DIR.mkdir(exist_ok=True)

@tool
def save_summary(summary_text: str, batch_id: int) -> str:
    """
    Сохраняет резюме батча в файл summaries/summary_batch_{id}.txt
    """
    p = SUMMARY_DIR / f"summary_batch_{batch_id}.txt"
    p.write_text(summary_text, encoding="utf-8")
    return f"saved_to={str(p)}"


In [48]:
from langchain.agents import create_agent
from langchain.agents.middleware import (
    PIIMiddleware,
    SummarizationMiddleware,
)
# from langchain_tavily import TavilySearch  # если понадобится веб-поиск

SYSTEM_PROMPT = """
Ты — аналитик переписки по проектам.
Твоя задача — обработать всю переписку порциями (батчами), создать
summary каждого батча и сохранить его в файл через инструмент save_summary.
Итоговый общий отчёт по проекту делать НЕ нужно — это будет выполнять другой агент.

========================================
ОБЩИЙ АЛГОРИТМ РАБОТЫ
========================================

Шаг 1. Определи project_hint из вопроса пользователя
(например, “Segezha”, код ЗНИ, ключевые слова и т.д.)

Шаг 2. Запускай цикл обработки батчей:

  2.1. Вызови search_project_emails_batch(project_hint, offset, batch_size=50)

  2.2. Получи результаты батча:
        - batch (список объектов)
        - offset
        - has_more

  2.3. Сгенерируй summary ТОЛЬКО для этого батча:
        - выдели ключевые темы
        - промежуточные выводы
        - что обсуждалось
        - без глобальных итогов

  2.4. Вызови save_summary(summary_text, batch_id)
        batch_id — порядковый номер батча, начиная с 1

  2.5. Если has_more = true:
          offset = offset + batch_size
          перейди к шагу 2.1
       Иначе:
          завершить цикл и сообщить пользователю «Все батчи обработаны»
          (но НЕ делай итогового отчёта)

========================================
ОГРАНИЧЕНИЯ
========================================
- НЕ делай глобального анализа.
- НЕ формируй общий отчёт по проекту.
- НЕ агрегируй темы между батчами.
- Каждый batch должен быть сохранён отдельно.
- Работай строго по циклу batch → summary → save → next batch.

========================================
ПРИМЕЧАНИЯ
========================================
- В summary описывай только то, что есть внутри текущего batch.
- Запрещено запрашивать инструмент search_project_emails.
- Использовать нужно ТОЛЬКО search_project_emails_batch и save_summary.


"""

agent = create_agent(
    # model="gpt-5",  # или твой модельный endpoint
    # model="claude-sonnet-4-20250514",  # или твой модельный endpoint
    model= "deepseek-reasoner",  # или твой модельный endpoint


    tools=[
        # search_project_emails,
        search_project_emails_batch,
        save_summary,
        search_emails_raw,   # опционально
    ],
    system_prompt=SYSTEM_PROMPT,
    middleware=[
        # скрываем e-mail и телефоны в вводе/выводе (настраиваем по желанию)
        PIIMiddleware("email", strategy="redact", apply_to_input=True),
        PIIMiddleware(
            "phone_number",
            detector=(
                r"(?:\+?\d{1,3}[\s.-]?)?"
                r"(?:\(?\d{2,4}\)?[\s.-]?)?"
                r"\d{3,4}[\s.-]?\d{4}"
            ),
            strategy="redact",
        ),
        # если переписка большая, middleware будет автоматически её подрезать и резюмировать
        SummarizationMiddleware(
            model="gpt-5",
            # model="claude-sonnet-4-20250514",  # или твой модельный endpoint
            max_tokens_before_summary=500,
        ),
    ],
)


In [25]:
result = agent.invoke({
    "messages": [
        {
            "role": "user",
            "content": """Начни обработку проекта Segezha батчами.
            Делай summary каждого батча и сохраняй их через save_summary.
            Итоговый отчёт НЕ делай. """

        }
    ]
})

report = result["messages"][-1].content
print(report)


Все батчи обработаны


### второй агент

In [26]:
from pathlib import Path
from langchain.tools import tool

SUMMARY_DIR = Path("summaries")
SUMMARY_DIR.mkdir(exist_ok=True)

@tool
def load_all_summaries() -> str:
    """
    Читает все файлы summary_batch_*.txt в каталоге summaries/.
    Возвращает один объединённый текст.
    """
    files = sorted(SUMMARY_DIR.glob("summary_batch_*.txt"))

    if not files:
        return "NO_SUMMARIES_FOUND"

    texts = []
    for file in files:
        content = file.read_text(encoding="utf-8")
        texts.append(f"===== {file.name} =====\n{content}\n")

    return "\n".join(texts)


In [27]:
from langchain.agents import create_agent
from langchain.agents.middleware import (
    PIIMiddleware,
    SummarizationMiddleware,
)
# from langchain_tavily import TavilySearch  # если понадобится веб-поиск

SYSTEM_PROMPT = """
Ты — эксперт по агрегированию больших массивов информации.

Твоя задача — взять summary отдельных батчей (их подготовил другой агент),
и создать единый, структурированный глобальный отчет.

Алгоритм:

1) Вызови load_all_summaries().
2) Проведи глубокий анализ:
   - ключевые темы,
   - решения,
   - риски,
   - нерешённые вопросы,
   - повторяющиеся инциденты.

3) Выдай ОДИН финальный отчёт:

Структура:
1. Краткое резюме по проекту.
2. Основные темы.
3. Ключевые решения.
4. Проблемы и риски.
5. Повторяющиеся инциденты (объединённо).
6. Финальное резюме.

Не переписывай батчи дословно — делай смысловое сжатие.

"""

agent_global = create_agent(
    model="gpt-5",  # или твой модельный endpoint
    # model="claude-sonnet-4-20250514",  # или твой модельный endpoint

    tools=[
        # search_project_emails,
        search_project_emails_batch,
        save_summary,
        load_all_summaries,
        search_emails_raw,   # опционально
    ],
    system_prompt=SYSTEM_PROMPT,
    middleware=[
        # скрываем e-mail и телефоны в вводе/выводе (настраиваем по желанию)
        PIIMiddleware("email", strategy="redact", apply_to_input=True),
        PIIMiddleware(
            "phone_number",
            detector=(
                r"(?:\+?\d{1,3}[\s.-]?)?"
                r"(?:\(?\d{2,4}\)?[\s.-]?)?"
                r"\d{3,4}[\s.-]?\d{4}"
            ),
            strategy="redact",
        ),
        # если переписка большая, middleware будет автоматически её подрезать и резюмировать
        SummarizationMiddleware(
            model="gpt-5",
            max_tokens_before_summary=1000,
        ),
    ],
)


In [28]:
result = agent_global.invoke({
    "messages": [
        {
            "role": "user",
            "content": """Начни обработку сделанных раннее summaries.
            Сделай итоговый отчет по всему проектy.
            """

        }
    ]
})

report = result["messages"][-1].content
print(report)


1. Краткое резюме по проекту
Проект фокусируется на стабилизации контуров логистики и выручки: корректность данных TM–SD (ЗНИ 272), устранение расхождений ВГО, доработка «Зеркало 2» по валютным счетам, перенос хранения номера вагона (ЗНИ 239), и внедрение распределения фактических транспортных затрат (TM→S4 с множественной контировкой). Параллельно идут работы по автоматизации EWM (автораспределение), согласованию регламентов и закрытию SD/JIRA-инцидентов. Критичны сроки к закрытию месяца; часть задач завязана на согласование ЗНИ и межблочную координацию.

2. Основные темы
- ЗНИ 272 (TM–SD, корректность данных для выручки)
  - Цель: передача даты коносамента и корректное заполнение даты перехода рисков; устранение типовых ошибок в заказах/поставках.
  - Статус: подтверждена срочность; SD-спека по дате перехода рисков не финализирована; предварительный горизонт внедрения — не ранее середины марта, общий — 1–2 месяца.
  - До релиза: ежедневный мониторинг новых заказов, ручное внесение фа

In [29]:
from pipeline import EmailSummaryPipeline

pipeline = EmailSummaryPipeline(qv=qv, model="gpt-5")

In [30]:
pipeline.run_batch_processing("Segezha")


{'messages': [HumanMessage(content='Here is a summary of the conversation to date:\n\n- Сохранено резюме Батч 1 (offset 0–49), период ~25–26.02.2021. Файл: summaries\\summary_batch_1.txt\n- Ключевые темы и действия:\n  - «Зеркало 2» (валютные счета): из ~20 док-ов создалось только 4 ПЗД на стороне ДЗК. Пример: ПЗД 5105630181 (БЕ 1650). В одном док-те несколько ДРФ; предложено отправлять по отдельности. Действия: прислать примеры/скриншоты; протестировать раздельную отправку ДРФ; подтвердить настройки.\n  - Расхождения по ВГО: ЗНИ 281 согласовано; план реализации ориентировочно 5 марта. Действия: подтвердить план-график и готовность к релизу.\n  - Корректность данных в SAP для расчёта выручки (ЗНИ 272): интеграция TM–SD, номер ЗНИ ещё не присвоен. Действия: присвоить/сообщить номер, согласовать сроки, получить вводные от финблока.\n  - SAPSP-17498 (автораспределение поставок по ж.д.): уведомления Jira SD. Действие: контролировать в SD-портале.\n  - Ж/д отгрузка, заведение ГУ12 (СЦБК): п

In [31]:
final = pipeline.run_global_summary("Segezha")
print(final["messages"][-1].content)


1. Краткое резюме по проекту
Проект «Segezha» в отчетный период (25–28.02.2021) сосредоточен на стабилизации контура логистика–сбыт–финансы: корректность выручки по МСФО 15 (TM→SD, ЗНИ 272), сверка ВГО (ЗНИ 281), доработка «Зеркала 2» (валютные счета и многократные ДРФ), автоматизация распределения в EWM (SAPSP-17498), железнодорожные сценарии (ГУ-12/ГУ-29, ЗНИ 239), регламенты ТМ и разнесение фактических транспортных затрат. Достигнут ряд технических результатов (фикс двойного налогообложения, запуск автораспределения для бумаги, закрытие инцидента Transporeon), при этом ключевые ЗНИ находятся в согласовании/реализации и требуют синхронизации по срокам и спецификациям.

2. Основные темы
- Корректность данных для выручки (ЗНИ 272, TM→SD): передача даты коносамента и даты перехода рисков, запуск оперативного мониторинга ошибок, план переноса в продуктив (ориентир — середина марта при доработке спецификации).
- Сверка ВГО (ЗНИ 281): методика «нарастающим итогом», урегулирование курсовых 

In [36]:
from pipeline import EmailSummaryPipeline

pipeline = EmailSummaryPipeline(qv=qv, model="gpt-5")


class Orchestrator:
    def __init__(self, pipeline):
        self.pipeline = pipeline
        self.run_batch_tool = tool(self.run_batch)
        self.run_global_tool = tool(self.run_global)

    def run_batch(self, project_hint: str) -> str:
        """
        Запуск batch-обработки.
        """
        result = self.pipeline.run_batch_processing(project_hint)
        # agent.invoke возвращает сложный объект → извлекаем текст
        return result["messages"][-1].content

    def run_global(self, project_hint: str) -> str:
        """
        Запуск глобальной агрегации.
        """
        result = self.pipeline.run_global_summary(project_hint)
        return result["messages"][-1].content


In [49]:
orchestrator = Orchestrator(pipeline)

ORCH_SYSTEM_PROMPT = """
Ты — главный управляющий агент.

Твоя задача:
- понять намерение пользователя
- выбрать подходящего подчинённого агента
- нужно вызывать по порядку:
          1.  run_batch(project_hint)  — произвести batch-сбор summary
          2.  run_global(project_hint) — сделать финальное summary

Пользователь может писать в свободной форме.
Ты должен:
  - определить project_hint
  - определить, что хочет пользователь: батчи или итоговый отчёт
  - вызвать нужные инструменты
  - вернуть пользователю результат

Не выполняй анализ почты сам — всё через инструменты.
"""

main_agent = create_agent(
    # model="gpt-5.1",
    model="deepseek-reasoner",
    tools=[
        orchestrator.run_batch_tool,
        orchestrator.run_global_tool,
    ],
    system_prompt=ORCH_SYSTEM_PROMPT,
)


In [50]:
result = main_agent.invoke({
    "messages": [
        {"role": "user", "content": "Сделай итоговый отчёт по проекту Segezha"}
    ]
})
print(result["messages"][-1].content)


Итоговый отчёт по проекту Segezha

1. Краткое резюме по проекту
Проект “Segezha” движется по нескольким ключевым трекам: исправление расхождений по ВГО (ЗНИ 281, целевая дата 5 марта), повышение корректности данных для выручки по МСФО 15 через интеграцию TM–SD (ЗНИ 272; согласовано по сути, реализация — не ранее середины марта, оценка 1–2 месяца), автоматизация распределения поставок в EWM (запущено 26.02, мониторятся исключения), а также доработки по “Зеркалу 2” для валютных счетов (правила по кодам валют и поддержка множественных ДРФ). Параллельно прорабатываются: разнесение фактических транспортных затрат (архитектура и трудозатраты), перенос хранения номера вагона (ЗНИ 239), запуск печати полного комплекта ГУ-29 для контейнеров, регламенты TM/ОП, учет вывозки сырья (ЗНИ-282), НСИ/интеграции и доступы. Основные риски — корректность учета выручки/валюты и операционные ограничения до завершения ключевых ЗНИ.

2. Основные темы
- TM–SD и МСФО 15 (ЗНИ 272):
  - Передача даты коносамента 

### вывод инфо

In [45]:
from openai import OpenAI
import os

# SDK автоматически возьмет DEEPSEEK_API_KEY если api_key=None
client = OpenAI(
    api_key=os.getenv("DEEPSEEK_API_KEY"),
    base_url="https://api.deepseek.com/v1"
)

response = client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": "Ты тестовый ассистент."},
        {"role": "user", "content": "Проверка соединения. Напиши 'OK'."}
    ]
)

print(response.choices[0].message.content)


OK
